In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
import numpy as np
import os

In [2]:
os.chdir("C:\Samita\Data science course\Python\Project\Consumer_Complaints")

In [3]:
cc_train = pd.read_csv("Consumer_Complaints_train.csv")
cc_test = pd.read_csv("Consumer_Complaints_test_share.csv")

In [4]:
cc_train.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572


In [5]:
cc_test.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID
0,2014-01-18,Bank account or service,Cashing a check without an account,Deposits and withdrawals,NaN,NaN,NaN,Bank of America,CA,95691,NaN,NaN,Web,2014-01-17,Closed with explanation,Yes,675956
1,2016-03-31,Debt collection,Credit card,Cont'd attempts collect debt not owed,Debt was paid,NaN,NaN,"National Credit Adjusters, LLC",FL,32086,NaN,Consent not provided,Web,2016-03-31,Closed with explanation,Yes,1858795
2,2012-03-08,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,94618,NaN,NaN,Web,2012-03-09,Closed without relief,Yes,32637
3,2016-01-07,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,Company chooses not to provide a public response,"TransUnion Intermediate Holdings, Inc.",FL,33584,Older American,NaN,Postal mail,2016-01-12,Closed with non-monetary relief,Yes,1731374
4,2013-08-23,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,FL,33543,NaN,NaN,Web,2013-08-23,Closed with explanation,Yes,501487


In [6]:
cc_test['Consumer disputed?'] = np.nan

In [7]:
cc_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 478421 non-null  object
 1   Product                       478421 non-null  object
 2   Sub-product                   339948 non-null  object
 3   Issue                         478421 non-null  object
 4   Sub-issue                     185796 non-null  object
 5   Consumer complaint narrative  75094 non-null   object
 6   Company public response       90392 non-null   object
 7   Company                       478421 non-null  object
 8   State                         474582 non-null  object
 9   ZIP code                      474573 non-null  object
 10  Tags                          67206 non-null   object
 11  Consumer consent provided?    135487 non-null  object
 12  Submitted via                 478421 non-null  object
 13 

In [8]:
cc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119606 entries, 0 to 119605
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Date received                 119606 non-null  object 
 1   Product                       119606 non-null  object 
 2   Sub-product                   84854 non-null   object 
 3   Issue                         119606 non-null  object 
 4   Sub-issue                     46546 non-null   object 
 5   Consumer complaint narrative  18557 non-null   object 
 6   Company public response       22776 non-null   object 
 7   Company                       119606 non-null  object 
 8   State                         118681 non-null  object 
 9   ZIP code                      118680 non-null  object 
 10  Tags                          16871 non-null   object 
 11  Consumer consent provided?    33864 non-null   object 
 12  Submitted via                 119605 non-nul

In [9]:
cc_train.isnull().sum()

Date received                        0
Product                              0
Sub-product                     138473
Issue                                0
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
Company                              0
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype: int64

In [10]:
cc_train.shape[0]

478421

In [11]:
len(cc_train['Consumer complaint narrative'].value_counts())

74019

In [12]:
len(cc_train['Consumer complaint narrative'].unique())

74020

In [13]:
len(cc_train['Product'].unique())

12

In [14]:
cc_train['data'] = 'train'
cc_test['data'] = 'test'

In [15]:
cc_train.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train


In [16]:
cc_all = pd.concat([cc_train,cc_test],axis = 0)

In [17]:
cc_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 598027 non-null  object
 1   Product                       598027 non-null  object
 2   Sub-product                   424802 non-null  object
 3   Issue                         598027 non-null  object
 4   Sub-issue                     232342 non-null  object
 5   Consumer complaint narrative  93651 non-null   object
 6   Company public response       113168 non-null  object
 7   Company                       598027 non-null  object
 8   State                         593263 non-null  object
 9   ZIP code                      593253 non-null  object
 10  Tags                          84077 non-null   object
 11  Consumer consent provided?    169351 non-null  object
 12  Submitted via                 598026 non-null  object
 13 

In [18]:
for col in ['Date sent to company','Date received']:
    cc_all[col] = pd.to_datetime(cc_all[col],infer_datetime_format=True)

In [19]:
cc_all.isnull().sum()

Date received                        0
Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
dtype: int64

In [20]:
cc_all['day_diff'] = pd.to_numeric(cc_all['Date sent to company'] - cc_all['Date received'])

In [21]:
cc_all.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data,day_diff
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train,86400000000000
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train,518400000000000
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train,1814400000000000
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train,0
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train,691200000000000


In [22]:
for col in ['Date sent to company','Date received']:
    cc_all.drop([col],axis=1,inplace=True)

In [23]:
for col in cc_all.select_dtypes(['object']).columns:
    print(col,':',cc_all[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 92118
Company public response : 10
Company : 3437
State : 62
ZIP code : 27342
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [24]:
cc_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
dtype: int64

In [25]:
for col in ['Company public response','Consumer complaint narrative',
            'Consumer consent provided?','Sub-issue',"Sub-product",'Tags']:
    varname = col.replace(' ','_').replace('?','_').replace('-','_') + '_isNaN'
    cc_all[varname] = np.where(pd.isnull(cc_all[col]),1,0)
    del cc_all[col]

In [26]:
cc_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 18 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Product                             598027 non-null  object
 1   Issue                               598027 non-null  object
 2   Company                             598027 non-null  object
 3   State                               593263 non-null  object
 4   ZIP code                            593253 non-null  object
 5   Submitted via                       598026 non-null  object
 6   Company response to consumer        598027 non-null  object
 7   Timely response?                    598027 non-null  object
 8   Consumer disputed?                  478421 non-null  object
 9   Complaint ID                        598027 non-null  int64 
 10  data                                598027 non-null  object
 11  day_diff                            598

In [27]:
cc_all.head()

,Product,Issue,Company,State,ZIP code,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data,day_diff,Company_public_response_isNaN,Consumer_complaint_narrative_isNaN,Consumer_consent_provided__isNaN,Sub_issue_isNaN,Sub_product_isNaN,Tags_isNaN
0,Credit card,Billing statement,Wells Fargo & Company,MI,48342,Web,Closed with explanation,Yes,No,856103,train,86400000000000,1,1,1,1,1,0
1,Bank account or service,"Making/receiving payments, sending money",Santander Bank US,PA,18042,Referral,Closed,Yes,No,1034666,train,518400000000000,1,1,1,1,0,1
2,Credit reporting,Incorrect information on credit report,Equifax,CA,92427,Referral,Closed with non-monetary relief,Yes,No,756363,train,1814400000000000,1,1,1,0,1,1
3,Credit card,Billing statement,U.S. Bancorp,GA,305XX,Web,Closed with monetary relief,Yes,No,1474177,train,0,0,0,0,1,1,0
4,Credit card,Transaction issue,Bank of America,MA,02127,Web,Closed with explanation,Yes,No,1132572,train,691200000000000,1,1,1,1,1,1


In [28]:
for col in cc_all.select_dtypes(['object']).columns:
    print(col,":",cc_all[col].nunique())

Product : 12
Issue : 95
Company : 3437
State : 62
ZIP code : 27342
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [29]:
for col in ['ZIP code','Company']:
    cc_all.drop([col],axis=1,inplace=True)

In [30]:
cc_all['Consumer disputed?'] = np.where(cc_all['Consumer disputed?']=='Yes',1,0)

In [31]:
k = cc_all['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname = 'Issue_'+val.replace(',','_').replace(' ','_')
    cc_all[varname] = np.where(cc_all['Issue']==val,1,0)

del cc_all['Issue']

In [32]:
k = cc_all['State'].value_counts()
for val in k.axes[0][0:10]:
    varname = 'State_'+val.replace(',','_').replace(' ','_')
    cc_all[varname] = np.where(cc_all['State']==val,1,0)

del cc_all['State']

In [33]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp = pd.get_dummies(cc_all[col],prefix=col,drop_first=True)
    cc_all = pd.concat([temp,cc_all],1)
    cc_all.drop([col],1,inplace=True)

In [34]:
for col in cc_all.select_dtypes(['object']).columns:
    print(col,':',cc_all[col].nunique())

data : 2


In [35]:
cc_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 53 columns):
 #   Column                                                        Non-Null Count   Dtype 
---  ------                                                        --------------   ----- 
 0   Timely response?_Yes                                          598027 non-null  uint8 
 1   Company response to consumer_Closed with explanation          598027 non-null  uint8 
 2   Company response to consumer_Closed with monetary relief      598027 non-null  uint8 
 3   Company response to consumer_Closed with non-monetary relief  598027 non-null  uint8 
 4   Company response to consumer_Closed with relief               598027 non-null  uint8 
 5   Company response to consumer_Closed without relief            598027 non-null  uint8 
 6   Company response to consumer_Untimely response                598027 non-null  uint8 
 7   Submitted via_Fax                                             598

In [36]:
cc_all['Consumer disputed?'].value_counts()

0    496596
1    101431
Name: Consumer disputed?, dtype: int64

In [37]:
cc_all.isnull().sum().sum()

0

In [38]:
train_clean = cc_all[cc_all['data']=='train']
del train_clean['data']
test_clean = cc_all[cc_all['data']=='test']
test_clean.drop(['data','Consumer disputed?'],axis=1,inplace=True)

In [39]:
train_clean.shape,test_clean.shape

((478421, 52), (119606, 51))

In [40]:
train_clean.head()

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_CA,State_FL,State_TX,State_NY,State_GA,State_NJ,State_PA,State_IL,State_VA,State_MD
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
X = train_clean.drop(['Consumer disputed?'],1)
y = train_clean['Consumer disputed?']
X.shape, y.shape

((478421, 51), (478421,))

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [43]:
X_train.shape,y_train.shape

((382736, 51), (382736,))

In [44]:
X_val.shape,y_val.shape

((95685, 51), (95685,))

In [45]:
from sklearn import tree

In [46]:
dtree_full=tree.DecisionTreeClassifier(criterion="entropy")

In [47]:
dtree_full.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [48]:
dtree=tree.DecisionTreeClassifier(criterion="entropy",
                                  max_leaf_nodes=10,
                                  class_weight="balanced")

In [49]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=10, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [50]:
predicted=dtree.predict(X_val)

df_test=pd.DataFrame(list(zip(y_val,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
k

predicted,0,1
real,,
0,39036,36505
1,7308,12836


In [53]:
from sklearn.metrics import roc_auc_score

In [54]:
df_test['Probs']=pd.DataFrame(dtree.predict_proba(X_val))[1]
print('AUC is :', roc_auc_score(df_test['real'],df_test['Probs']))


AUC is : 0.6105998569476903


In [51]:
import os

In [52]:
submission=pd.DataFrame(list(zip(test_clean['Complaint ID'],list(predicted))),
columns=['Complaint ID','Consumer disputed?'])
submission.to_csv('R_P1_1.csv',index=False)

In [53]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)
print('Precision is :', TP/(TP + FP))

12836 39036 36505 7308
Accuracy is : 0.5421121387887339
Sensitivity is : 0.6372120730738682
Specificity is : 0.5167524920241988
Precision is : 0.26014876066557224
